In [ ]:
!pip install transformers datasets scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [ ]:
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import (
    BertTokenizerFast, BertForSequenceClassification,
    get_scheduler, DataCollatorWithPadding
)
from datasets import load_dataset
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns



In [ ]:
dataset = load_dataset("imdb")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

# Keep only 'text' and 'label' columns to avoid errors
dataset = dataset.remove_columns([col for col in dataset["train"].column_names if col not in ["text", "label"]])

# Tokenize the text column with padding & truncation
def tokenize_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Set format to PyTorch tensors
tokenized_datasets.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

train_dataset = tokenized_datasets["train"]
test_dataset = tokenized_datasets["test"]




tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=data_collator)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=data_collator)




In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)



Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer_grouped_parameters = [
    {"params": model.bert.embeddings.parameters(), "lr": 5e-5},
    {"params": model.bert.encoder.layer[:6].parameters(), "lr": 3e-5},
    {"params": model.bert.encoder.layer[6:].parameters(), "lr": 1e-5},
    {"params": model.classifier.parameters(), "lr": 1e-4}
]

optimizer = AdamW(optimizer_grouped_parameters, weight_decay=0.01)

num_training_steps = len(train_loader) * 3  # 3 epochs
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)



In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def train():
    model.train()
    for epoch in range(3):
        loop = tqdm(train_loader, leave=True)
        for batch in loop:
            batch = {k: v.to(device) for k, v in batch.items() if k in ["input_ids", "attention_mask", "labels"]}
            outputs = model(**batch)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            loop.set_description(f"Epoch {epoch+1}")
            loop.set_postfix(loss=loss.item())



In [ ]:
def evaluate():
    model.eval()
    predictions, labels, texts = [], [], []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            label_ids = batch["labels"].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=label_ids)
            logits = outputs.logits
            preds = torch.argmax(logits, axis=-1)
            predictions.extend(preds.cpu().numpy())
            labels.extend(label_ids.cpu().numpy())
            texts.extend(tokenizer.batch_decode(input_ids, skip_special_tokens=True))
            if len(texts) >= 5:  # collect only first few for sample output
                break

    # Show sample outputs
    print("\n📌 Top 5 Sample Predictions:")
    for text, pred, label in zip(texts[:5], predictions[:5], labels[:5]):
        print(f"\n📝 Text: {text[:120]}...")
        print(f"🔮 Predicted Label: {pred} | 🎯 Actual Label: {label}")

    # Accuracy, Error Rate, F1
    acc = accuracy_score(labels, predictions)
    error_rate = 1 - acc
    f1 = f1_score(labels, predictions, average="weighted")
    print(f"\n✅ Accuracy    : {acc*100:.2f}%")
    print(f"❌ Error Rate : {error_rate*100:.2f}%")
    print(f"📊 F1 Score   : {f1*100:.2f}%")


In [ ]:
def tokenize_function(examples):
    # Ensure 'text' is a list of strings before tokenization
    texts = [str(text) for text in examples["text"]]  # Convert 'text' to string explicitly
    return tokenizer(texts, padding=True, truncation=True)

In [ ]:
train()
evaluate()



Epoch 3: 100%|██████████| 1563/1563 [37:34<00:00,  1.44s/it, loss=0.0634]



📌 Top 5 Sample Predictions:

📝 Text: i love sci - fi and am willing to put up with a lot. sci - fi movies / tv are usually underfunded, under - appreciated a...
🔮 Predicted Label: 0 | 🎯 Actual Label: 0

📝 Text: worth the entertainment value of a rental, especially if you like action movies. this one features the usual car chases,...
🔮 Predicted Label: 0 | 🎯 Actual Label: 0

📝 Text: its a totally average film with a few semi - alright action sequences that make the plot seem a little better and remind...
🔮 Predicted Label: 0 | 🎯 Actual Label: 0

📝 Text: star rating : * * * * * saturday night * * * * friday night * * * friday morning * * sunday night * monday morning < br ...
🔮 Predicted Label: 0 | 🎯 Actual Label: 0

📝 Text: first off let me say, if you haven ' t enjoyed a van damme movie since bloodsport, you probably will not like this movie...
🔮 Predicted Label: 1 | 🎯 Actual Label: 0

✅ Accuracy    : 93.75%
❌ Error Rate : 6.25%
📊 F1 Score   : 96.77%
